In [1]:
import geopandas as gpd
from geoutils import  add_feature
from pathlib import Path
import glob
import pandas as pd
import numpy as np

In [2]:
ifn4_dir = 'ifn4_biomass_flights/'
mfe_dir = '/home/dibepa/Documents/MFESpain/'

In [3]:
for mfe_file in Path(mfe_dir).glob('MFE*.shp'):
    print(mfe_file.stem)
    if not Path(f'{mfe_dir}/dissolved_{mfe_file.stem}.shp').exists():
        
        mfe_contour = gpd.read_file(mfe_file).dissolve()
        mfe_contour.to_file(f'{mfe_dir}/dissolved_{mfe_file.stem}.shp',driver='ESRI Shapefile',index=False)
    

MFE_43
MFE_62
MFE_30
MFE_13
MFE_52
MFE_42
MFE_12
MFE_11
MFE_22
MFE_23
MFE_24
MFE_61
MFE_51
MFE_41
MFE_21


In [4]:

for ifn4_file in Path(ifn4_dir).glob('*.shp'):
    print(ifn4_file.stem)
    ifn4_points = gpd.read_file(ifn4_file)
    original_crs = ifn4_points.crs
    ifn4_allometry = gpd.GeoDataFrame()

    for mfe_file in Path(mfe_dir).glob('MFE*.shp'):
        mfe_contour = gpd.read_file(f'{mfe_dir}dissolved_{mfe_file.stem}.shp')
        crs = mfe_contour.crs
        ifn4_points = ifn4_points.to_crs(mfe_contour.crs)

        intersecting = ifn4_points.geometry.intersects(mfe_contour.iloc[0].geometry)

        if np.any(intersecting):
            print('\nFound a matching CCAA\n')
            gdf = ifn4_points[intersecting]
            gdf = add_feature(gdf,'FormArbol',mfe_file,col_name='Plot type')
            gdf = gdf.to_crs(original_crs)
            ifn4_allometry = pd.concat([ifn4_allometry,gdf])
    if len(ifn4_allometry.index)>0:    
        print('Saving')
        ifn4_allometry.to_file(f'ifn4_biomass_plottype/{ifn4_file.stem}.shp',driver='ESRI Shapefile',index=False)

ifn4_29_biomass_flight

Found a matching CCAA


Found a matching CCAA

Saving
ifn4_30_biomass_flight

Found a matching CCAA


Found a matching CCAA


Found a matching CCAA

Saving
ifn4_31_biomass_flight

Found a matching CCAA

Saving


In [6]:
ifn4_allometry.columns

Index(['index', 'Estadillo', 'Provincia', 'AGC', 'BGC', 'AGB', 'BGB', 'Type',
       'Year', 'Region', 'pnoa_file', 'mission', 'geometry', 'Plot type'],
      dtype='object')